## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cherskiy,RU,68.7500,161.3000,-43.10,98,46,3.51,scattered clouds
1,1,Port Alfred,ZA,-33.5906,26.8910,72.57,92,88,4.05,overcast clouds
2,2,Pisco,PE,-13.7000,-76.2167,70.02,73,100,19.57,mist
3,3,Mahebourg,MU,-20.4081,57.7000,71.26,73,37,9.89,light rain
4,4,Avarua,CK,-21.2078,-159.7750,82.45,83,75,12.66,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                163
City                   163
Country                162
Lat                    163
Lng                    163
Max Temp               163
Humidity               163
Cloudiness             163
Wind Speed             163
Current Description    163
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna(how='any')

In [8]:
# Verify empty rows have been dropped
preferred_cities_df.count()

City_ID                162
City                   162
Country                162
Lat                    162
Lng                    162
Max Temp               162
Humidity               162
Cloudiness             162
Wind Speed             162
Current Description    162
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,
10,Codrington,AU,89.44,clear sky,-38.2667,141.9667,
16,Lazaro Cardenas,MX,77.22,scattered clouds,17.9583,-102.2000,
18,Rikitea,PF,78.60,clear sky,-23.1203,-134.9692,
21,Nelson Bay,AU,83.28,clear sky,-32.7167,152.1500,
24,Kuching,MY,75.42,fog,1.5500,110.3333,
30,Bengkulu,ID,78.93,overcast clouds,-3.8004,102.2655,
31,Harper,LR,79.03,few clouds,4.3750,-7.7169,
46,Poum,NC,80.98,few clouds,-20.2333,164.0167,
47,Beira,MZ,80.69,scattered clouds,-19.8436,34.8389,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        pass    

In [13]:
# Display DataFrame with hotel names
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,Paradise Inn
10,Codrington,AU,89.44,clear sky,-38.2667,141.9667,Codrington Gardens Bed And Breakfast
16,Lazaro Cardenas,MX,77.22,scattered clouds,17.9583,-102.2000,Baymont by Wyndham Lazaro Cardenas
18,Rikitea,PF,78.60,clear sky,-23.1203,-134.9692,People ThankYou
21,Nelson Bay,AU,83.28,clear sky,-32.7167,152.1500,Mantra Nelson Bay


In [14]:
# Check for empty rows
hotel_df.count()

City                   162
Country                162
Max Temp               162
Current Description    162
Lat                    162
Lng                    162
Hotel Name             162
dtype: int64

In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))